In [1]:
import pyotp
import configparser
from smartapi import SmartConnect
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import math
from datetime import *
import re
from time import sleep
from time import time as tim
from bs4 import BeautifulSoup
from IPython.display import clear_output
import nsepython
import concurrent

config = configparser.ConfigParser()
config.read('nishad_credentials.ini')
client_id = config['angel']['client_id']
password = config['angel']['password']
api_key = config['angel']['api_key']
token = config['angel']['token']

def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = 'test inside fun'
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})

# fetch token/instrument from dataframe
def getTokenInfo (symbol,exch_seg='NSE',instrumenttype='OPTIDX',strike_price='',pe_ce='CE',expiry=None):
    df = token_df
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])
    

def today_is_a_trading_holiday():
    response = requests.get('https://zerodha.com/marketintel/holiday-calendar/')
    soup = BeautifulSoup(response.text, 'html.parser')
    soup = soup.find_all(name='h4')[0]
    holiday = soup.find(string=re.compile("The next trading holiday"))
    next_holiday = '-'.join(re.findall('([0-3][0-9]) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)',holiday)[0])
    next_holiday = next_holiday+'-'+str(datetime.now().year)
    next_holiday = datetime.strptime(next_holiday,'%d-%b-%Y').date()
    return next_holiday == datetime.today().date()

def place_order(token,symbol,qty,exch_seg,buy_sell,ordertype,triggerprice,price,variety,producttype):
    orderparams = {
            "variety": variety,
            "tradingsymbol": symbol,
            "symboltoken": token,
            "transactiontype": buy_sell,
            "exchange": exch_seg,
            "ordertype": ordertype,
            "producttype": producttype,
            "duration": "DAY",
            "price": price,
            "triggerprice":triggerprice, 
            "variety":variety,
            "squareoff": "0",
            "stoploss": "0",
            "quantity": qty
            }
    try:
        orderId=obj.placeOrder(orderparams)
        print("The order id is: {}".format(orderId))
        return orderId
    except Exception as e:
        print("Order placement failed: {}".format(e))

        

intializeSymbolTokenMap()
obj=SmartConnect(api_key=api_key)
data = obj.generateSession(client_id,password,pyotp.TOTP(token).now())
refreshToken= data['data']['refreshToken']
feedToken=obj.getfeedToken()
userProfile= obj.getProfile(refreshToken)
print(userProfile)


def exit_all_position():
    
    # cancel pendings orders
    try:
        orderbook = pd.DataFrame(obj.orderBook()['data'])
    except Exception as e:
            try:
                sleep(1)
                orderbook = pd.DataFrame(obj.orderBook()['data'])
            except Exception as e:
                print("Position fetch failed: {}".format(e))
    
    filt = orderbook['status'] == 'trigger pending'
    orderids = orderbook[filt]['orderid'].tolist()
    if orderids:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(cancel_order, orderids)
    
    # exit positions
    try:
        positions = pd.DataFrame(obj.position()['data'])
    except Exception as e:
            try:
                sleep(1)
                positions = pd.DataFrame(obj.position()['data'])
            except Exception as e:
                print("Position fetch failed: {}".format(e))
        
    filt = positions['netqty'] < '0'
    symboltoken = positions.loc[filt,['symboltoken','tradingsymbol','netqty','exchange','producttype']]

    trade_list = [] #empty list

    for index, row in symboltoken.iterrows():
        list_item = {"variety": 'NORMAL', 
                "tradingsymbol" : row['tradingsymbol'],
                "symboltoken" : row['symboltoken'],
                "transactiontype": 'BUY', 
                "exchange": row['exchange'],
                "ordertype": 'MARKET', 
                "producttype": row['producttype'],
                "duration": 'DAY', 
                "price": '0', 
                "quantity": abs(int(row['netqty'])),
                "triggerprice": '0'}

        trade_list.append(list_item)

    x = place_multiple_orders(trade_list)

    filt = positions['netqty'] > '0'
    symboltoken = positions.loc[filt,['symboltoken','tradingsymbol','netqty','exchange','producttype']]

    trade_list = [] #empty list

    for index, row in symboltoken.iterrows():
        list_item = {"variety": 'NORMAL', 
                "tradingsymbol" : row['tradingsymbol'],
                "symboltoken" : row['symboltoken'],
                "transactiontype": 'SELL', 
                "exchange": row['exchange'],
                "ordertype": 'MARKET', 
                "producttype": row['producttype'],
                "duration": 'DAY', 
                "price": '0', 
                "quantity": abs(int(row['netqty'])),
                "triggerprice": '0'}

        trade_list.append(list_item)

    y = place_multiple_orders(trade_list)

    print(x,y)     

def cancel_order(orderid):
    try:
        obj.cancelOrder(orderid,'STOPLOSS')
    except Exception as e:
        try:
            sleep(0.5)
            obj.cancelOrder(orderid,'STOPLOSS')
        except Exception as e:
            try:
                sleep(1)
                obj.cancelOrder(orderid,'STOPLOSS')
            except Exception as e:
                print("Order placement failed: {}".format(e))

def place_order_faster(orderparams):
    try:
        orderID = obj.placeOrder(orderparams)
        exchange = orderparams['exchange']
        symbol = orderparams['tradingsymbol']
        spot_token = orderparams['symboltoken']
        print("The order id is: {}".format(orderID))
        ltp_data = obj.ltpData(exchange,symbol,spot_token)['data']['ltp']
        return orderID,ltp_data
    except Exception as e:  #1st error
        sleep(1) #ensure you have imported time at the top
        try:
            orderID = obj.placeOrder(orderparams)
            exchange = orderparams['exchange']
            symbol = orderparams['tradingsymbol']
            spot_token = orderparams['symboltoken']
            print("The order id is: {}".format(orderID))
            ltp_data = obj.ltpData(exchange,symbol,spot_token)['data']['ltp']
            return orderID, ltp_data
        except Exception as e: #2nd error
            sleep(1)
            try:
                orderID = obj.placeOrder(orderparams)
                exchange = orderparams['exchange']
                symbol = orderparams['tradingsymbol']
                spot_token = orderparams['symboltoken']
                sleep(1)
                print("The order id is: {}".format(orderID))
                ltp_data = obj.ltpData(exchange,symbol,spot_token)['data']['ltp']
                return orderID, ltp_data
            except Exception as e:
                sleep(2)
                try:
                    orderID = obj.placeOrder(orderparams)
                    exchange = orderparams['exchange']
                    symbol = orderparams['tradingsymbol']
                    spot_token = orderparams['symboltoken']
                    sleep(2)
                    print("The order id is: {}".format(orderID))
                    ltp_data = obj.ltpData(exchange,symbol,spot_token)['data']['ltp']
                    return orderID, ltp_data
                except Exception as e:
                    print("Order placement failed: {}".format(e))

def place_multiple_orders(tradeList):
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = executor.map(place_order_faster, tradeList)
        values = []
        for value in results:
            values.append(value)
        return values

def position_digger(token_1,token_2):
    global pos_ce_qty,pos_pe_qty,mtm
    pos_ce_qty,pos_pe_qty,mtm = None, None, 0
    try:
        positions = pd.DataFrame(obj.position()['data'])
        mtm = positions['pnl'].astype(float).sum()
        filt1 = (positions['symboltoken'] == token_1)
        pos_ce_qty =  int(positions[filt1]['netqty'].iloc[0])
        filt2 = (positions['symboltoken'] == token_2)
        pos_pe_qty =  int(positions[filt2]['netqty'].iloc[0])
    except Exception as e:
        try:
            sleep(1)
            positions = pd.DataFrame(obj.position()['data'])
            mtm = positions['pnl'].astype(float).sum()
            filt1 = (positions['symboltoken'] == token_1)
            pos_ce_qty =  int(positions[filt1]['netqty'].iloc[0])
            filt2 = (positions['symboltoken'] == token_2)
            pos_pe_qty =  int(positions[filt2]['netqty'].iloc[0])
        except Exception as e:
            try:
                sleep(2)
                positions = pd.DataFrame(obj.position()['data'])
                mtm = positions['pnl'].astype(float).sum()
                filt1 = (positions['symboltoken'] == token_1)
                pos_ce_qty =  int(positions[filt1]['netqty'].iloc[0])
                filt2 = (positions['symboltoken'] == token_2)
                pos_pe_qty =  int(positions[filt2]['netqty'].iloc[0])
            except Exception as e:
                print("position digger failed: {}".format(e))

def reentry(orderprice,tradingsymbol,symboltoken,exchange,qty,stock_reentry,producttype):
    
    prev_order_avg = orderprice    
    latest_candle_close = obj.ltpData(exchange,tradingsymbol,symboltoken)['data']['ltp']
    
    if latest_candle_close < prev_order_avg:
        print(f'latest candle : {latest_candle_close} ,previous candle : {prev_order_avg}')
        orderparams = {"variety": 'NORMAL', 
                    "tradingsymbol" : tradingsymbol,
                    "symboltoken" : symboltoken,
                    "transactiontype": 'SELL', 
                    "exchange": exchange,
                    "ordertype": 'MARKET', 
                    "producttype": producttype,
                    "duration": 'DAY', 
                    "price": '0', 
                    "quantity": qty,
                    "triggerprice": '0'}
        values = place_order_faster(orderparams)
        orderid, orderprice  = values
        
        try:
            sleep(1)
            order_status = list(filter(lambda orderbook: orderbook['orderid'] == orderid, obj.orderBook()['data']))[0]['status']
            print(order_status)
            print(orderprice)

            if order_status == 'complete':

                sl_trigger = round(int(orderprice) * sl,2)
                sleep(0.5)
                sl_orderid = place_order(symboltoken,tradingsymbol,qty,exchange,'BUY','STOPLOSS_LIMIT',sl_trigger,sl_trigger+5,"STOPLOSS",producttype)
                stock_reentry -= 1
                print(f'available reentry for {tradingsymbol} : {stock_reentry}')
        
        except IndexError:
            print('order might not filled')

    return orderprice, stock_reentry

{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'A471478', 'name': 'ABDULLA NISHAD', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


### inputs

In [2]:
weekday_today = datetime.now().weekday()
# monday
if weekday_today == 0:
    slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s = 49,2,2,9,20,0
    exit_h,exit_m,exit_s,max_mtm,quantity = 13,50,0,-4600,50
# tuesday
elif weekday_today == 1:
    moneyness = 0
    slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s = 45,1,1,9,20,0
    exit_h,exit_m,exit_s,max_mtm,quantity = 14,50,0,-2700,25
# wednesday
elif weekday_today == 2:
    slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s = 53,3,3,9,20,0
    exit_h,exit_m,exit_s,max_mtm,quantity = 13,20,0,-6200,50
# thursday
elif weekday_today == 3:
    slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s = 45,1,1,9,20,0
    exit_h,exit_m,exit_s,max_mtm,quantity = 15,20,0,-5400,50
# friday
elif weekday_today == 4:
    slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s = 21,3,3,9,20,0
    exit_h,exit_m,exit_s,max_mtm,quantity = 14,20,0,-1800,50
    
print(slpercentage,ce_reentry,pe_reentry,entry_h,entry_m,entry_s,exit_h,exit_m,exit_s,max_mtm,quantity)

45 1 1 9 20 0 14 50 0 -2700 25


In [3]:
while datetime.now().time() < time(entry_h,entry_m-1,entry_s+55):
    sleep(1)

producttype = "INTRADAY"            
if weekday_today == 3:
    producttype = "CARRYFORWARD"
symbol = 'BANKNIFTY'

sl = (slpercentage + 100) / 100

payload = nsepython.nse_optionchain_scrapper(symbol)
currentExpiry,dte = nsepython.nse_expirydetails(payload,0)
spot_token = getTokenInfo(symbol).iloc[0]['token']
spot_ltp_data = obj.ltpData('NSE',symbol,spot_token)['data']['ltp']

if symbol == 'BANKNIFTY':
    atm_strike = 100 * round(spot_ltp_data/100)
    hedge_ce_points = 1800
    hedge_pe_points = -1800
elif symbol == 'NIFTY':
    atm_strike = 50 * round(spot_ltp_data/50)
    hedge_ce_points = 700
    hedge_pe_points = -700

#     if (weekday_today == 2)|(weekday_today == 3):  
#         quantity = quantity * 2
#         max_mtm = max_mtm * 2
#         hedge_ce = getTokenInfo(symbol,'NFO','OPTIDX',atm_strike+hedge_ce_points,'CE',currentExpiry).iloc[0]
#         hegde_ce_symbol  = hedge_ce['symbol']
#         hegde_ce_token = hedge_ce['token']
#         hegde_ce_qty = quantity
#         hegde_ce_exch = hedge_ce['exch_seg']
#         sleep(0.4)
#         hegde_ce_orderid = place_order(hegde_ce_token,hegde_ce_symbol,hegde_ce_qty,'NFO','BUY','MARKET',0,0,"NORMAL",producttype)
#         hedge_pe = getTokenInfo(symbol,'NFO','OPTIDX',atm_strike+hedge_pe_points,'PE',currentExpiry).iloc[0]
#         hedge_pe_symbol  = hedge_pe['symbol']
#         hedge_pe_token = hedge_pe['token']
#         hedge_pe_qty = quantity
#         hedge_pe_exch = hedge_pe['exch_seg']
#         sleep(0.4)
#         hedge_pe_orderid = place_order(hedge_pe_token,hedge_pe_symbol,hedge_pe_qty,'NFO','BUY','MARKET',0,0,"NORMAL",producttype)

atm_ce = getTokenInfo(symbol,'NFO','OPTIDX',atm_strike+moneyness,'CE',currentExpiry).iloc[0]
ce_symbol,ce_token,ce_qty,ce_exch  = atm_ce['symbol'],atm_ce['token'],quantity,atm_ce['exch_seg']
atm_pe = getTokenInfo(symbol,'NFO','OPTIDX',atm_strike-moneyness,'PE',currentExpiry).iloc[0]
pe_symbol,pe_token,pe_qty,pe_exch  = atm_pe['symbol'],atm_pe['token'],quantity,atm_pe['exch_seg']

trade_list = [] #empty list

ce_dict = {"variety": 'NORMAL', 
            "tradingsymbol" : ce_symbol,
            "symboltoken" : ce_token,
            "transactiontype": 'SELL', 
            "exchange": ce_exch,
            "ordertype": 'MARKET', 
            "producttype": producttype,
            "duration": 'DAY', 
            "price": '0', 
            "quantity": ce_qty,
            "triggerprice": '0'}

trade_list.append(ce_dict)

pe_dict = {"variety": 'NORMAL', 
            "tradingsymbol" : pe_symbol,
            "symboltoken" : pe_token,
            "transactiontype": 'SELL', 
            "exchange": ce_exch,
            "ordertype": 'MARKET', 
            "producttype": producttype,
            "duration": 'DAY', 
            "price": '0', 
            "quantity": ce_qty,
            "triggerprice": '0'}

trade_list.append(pe_dict)

while datetime.now().time() < time(entry_h,entry_m,entry_s):
    sleep(1)

start = datetime.now()
values = place_multiple_orders(trade_list)
end = datetime.now()
print(end - start)

ce_values, pe_values = values

ce_orderid, ce_orderprice = ce_values
pe_orderid, pe_orderprice = pe_values

try:
    sleep(1)
    ce = list(filter(lambda orderbook: orderbook['orderid'] == ce_orderid, obj.orderBook()['data']))[0]['status']
    sleep(1)
    pe = list(filter(lambda orderbook: orderbook['orderid'] == ce_orderid, obj.orderBook()['data']))[0]['status']
    print(ce,'\n',pe)

    if ce == 'complete' and pe == 'complete':
        ce_sl_trigger = round(int(ce_orderprice) * sl,1)
        sleep(0.5)
        ce_sl_orderid = place_order(ce_token,ce_symbol,ce_qty,'NFO','BUY','STOPLOSS_LIMIT',ce_sl_trigger,ce_sl_trigger+5,"STOPLOSS",producttype)
        pe_sl_trigger = round(int(pe_orderprice) * sl,1)
        sleep(0.5)
        pe_sl_orderid = place_order(pe_token,pe_symbol,pe_qty,'NFO','BUY','STOPLOSS_LIMIT',pe_sl_trigger,pe_sl_trigger+5,"STOPLOSS",producttype)

        print(f"ce_symbol='{ce_symbol}'\nce_token='{ce_token}'\nce_qty={ce_qty}")
        print(f"ce_exch='{ce_exch}'\nce_orderprice={ce_orderprice}\nce_reentry={ce_reentry}\n")
        print(f"pe_symbol='{pe_symbol}'\npe_token='{pe_token}'\npe_qty={pe_qty}")
        print(f"pe_exch='{pe_exch}'\npe_orderprice={pe_orderprice}\npe_reentry={pe_reentry}")
        print(f"\nmax_mtm={max_mtm}")
        print(f"producttype = '{producttype}'\nsl = {sl}")

    else:
        exit_all_position()

except IndexError:
    print('order might not filled')

The order id is: 230207000035086
The order id is: 230207000035090
0:00:01.821823
complete 
 complete
The order id is: 230207000035537
The order id is: 230207000035694
ce_symbol='BANKNIFTY09FEB2341500CE'
ce_token='39057'
ce_qty=25
ce_exch='NFO'
ce_orderprice=322.7
ce_reentry=1

pe_symbol='BANKNIFTY09FEB2341500PE'
pe_token='39058'
pe_qty=25
pe_exch='NFO'
pe_orderprice=335.95
pe_reentry=1

max_mtm=-2700
producttype = 'INTRADAY'
sl = 1.45


In [4]:
while datetime.now().time() < time(exit_h,exit_m,exit_s):
    
    position_digger(ce_token,pe_token)

    if mtm <= max_mtm:
        break
    else:
        if datetime.now().second == 0 or datetime.now().second == 1:
            if pos_ce_qty == 0 and ce_reentry > 0:
                ce_orderprice, ce_reentry = reentry(ce_orderprice,ce_symbol,ce_token,ce_exch,ce_qty,ce_reentry,producttype)
            if pos_pe_qty == 0 and pe_reentry > 0:
                pe_orderprice, pe_reentry = reentry(pe_orderprice,pe_symbol,pe_token,pe_exch,pe_qty,pe_reentry,producttype)

print('exiting all position')
exit_all_position()

KeyboardInterrupt: 

In [5]:
exit_all_position()

The order id is: 230207000207755
The order id is: 230207000207760
[('230207000207755', 298.0), ('230207000207760', 326.8)] []
